# OPI Parameter Fitting - Jupyter Notebook

This notebook demonstrates parameter optimization using observational data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '..')

import opi
from opi.io import create_synthetic_samples

print("Setup complete!")

## 1. Create Synthetic Observational Data

In real applications, this would be your measured isotope samples.

In [ ]:
# Create synthetic topography
dem = opi.create_synthetic_dem(
    topo_type='gaussian',
    grid_size=(300e3, 300e3),
    grid_spacing=(3000, 3000),
    lon0=0, lat0=45,
    amplitude=1500,
    sigma=(40e3, 40e3)
)

# Generate synthetic sample points along a transect
n_samples = 10
sample_x = np.linspace(-100000, 100000, n_samples)
sample_y = np.zeros(n_samples)

# Get elevations at sample points
from scipy.interpolate import RegularGridInterpolator
interp = RegularGridInterpolator(
    (dem['y'], dem['x']), dem['hGrid'], 
    bounds_error=False, fill_value=0
)
sample_elev = interp(np.column_stack([sample_y, sample_x]))

# Create synthetic isotope data (depletion with elevation)
# In real case, these would be your measured values
np.random.seed(42)
noise = np.random.normal(0, 5, n_samples)  # 5 permil noise

sample_d2h = (-80 - 0.02 * sample_elev + noise) * 1e-3  # fraction
sample_d18o = sample_d2h / 8 + np.random.normal(0, 0.5, n_samples) * 1e-3

print(f"Created {n_samples} synthetic samples")
print(f"Elevation range: {sample_elev.min():.0f} - {sample_elev.max():.0f} m")
print(f"d2H range: {sample_d2h.min()*1000:.1f} - {sample_d2h.max()*1000:.1f} permil")

In [ ]:
# Visualize sample locations
fig, ax = plt.subplots(figsize=(12, 5))

# Plot topography
im = ax.pcolormesh(dem['lon'], dem['lat'], dem['hGrid'], 
                   cmap='terrain', shading='auto')
plt.colorbar(im, ax=ax, label='Elevation (m)')

# Plot samples
lon_samp, lat_samp = opi.xy2lonlat(sample_x, sample_y, 0, 45)
scatter = ax.scatter(lon_samp, lat_samp, c=sample_d2h*1000, 
                    cmap='coolwarm_r', s=100, edgecolors='black')
plt.colorbar(scatter, ax=ax, label='d2H (permil)')

ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Sample Locations')
plt.show()

## 2. Create Run File for Fitting

In [ ]:
# Save sample data to Excel (for realistic workflow)
import pandas as pd

samples_df = pd.DataFrame({
    'line': range(1, n_samples+1),
    'longitude': lon_samp,
    'latitude': lat_samp,
    'elevation': sample_elev,
    'd2H': sample_d2h * 1000,  # Convert to permil for Excel
    'd18O': sample_d18o * 1000,
    'd_excess': (sample_d2h - 8*sample_d18o) * 1000,
    'sample_type': ['C'] * n_samples  # C for catchment
})

# Save
samples_df.to_excel('../examples/data/synthetic_samples.xlsx', index=False)
print("Sample data saved to synthetic_samples.xlsx")
samples_df.head()

In [ ]:
from opi.io import write_run_file
import os

# Create run file configuration
run_data = {
    'run_title': 'Synthetic Gaussian Mountain Fitting',
    'is_parallel': False,
    'data_path': os.path.abspath('../examples/data'),
    'topo_file': 'gaussian_topo.mat',
    'r_tukey': 0.25,
    'sample_file': 'synthetic_samples.xlsx',  # Your data file
    'cont_divide_file': None,
    'restart_file': None,
    'map_limits': [-2, 2, 43, 47],
    'section_lon0': None,
    'section_lat0': None,
    'mu': 25,           # CRS3 population factor
    'epsilon0': 1e-6,   # Stopping criterion
    'parameter_labels': ['U|azimuth|T0|M|kappa|tau_c|d2h0|d_d2h0_d_lat|f_p0'],
    'exponents': [0, 0, 0, 0, 0, 0, 3, 3, 0],
    # Parameter bounds: [U, az, T0, M, kappa, tau_c, d2h0, d_d2h0_d_lat, f_p0]
    'l_b': [0.1, -30, 265, 0, 0, 0, -15e-3, -5e-3, 0.5],
    'u_b': [25, 145, 295, 1.2, 1e6, 2500, 15e-3, 5e-3, 1.0],
    'beta': None  # No initial solution - will be fitted
}

# Write run file
write_run_file('../examples/data/fitting_run.run', run_data)
print("Run file created: fitting_run.run")

## 3. Run Parameter Fitting

In [ ]:
print("Starting parameter fitting...")
print("This may take a few minutes...\n")

# Run fitting (limited iterations for demo)
result = opi.opi_fit_one_wind(
    run_file_path='../examples/data/fitting_run.run',
    verbose=True,
    max_iterations=500  # Limit for demo (use 10000+ for real fitting)
)

print("\n" + "="*50)
print("FITTING COMPLETE")
print("="*50)

In [ ]:
# Display results
print(f"\nConvergence: {result['convergence']}")
print(f"Final misfit: {result['misfit']:.6f}")
print(f"Iterations: {result['iterations']}")

print("\nFitted Parameters:")
for name, value in result['solution_params'].items():
    print(f"  {name:20s}: {value:12.6f}")

## 4. Visualize Fitting Results

In [ ]:
# Run forward calculation with fitted parameters
fitted_beta = np.array(list(result['solution_params'].values()))

# Recalculate with fitted parameters
from opi.calc_one_wind import calc_one_wind

fitted_result = calc_one_wind(
    beta=fitted_beta,
    f_c=1e-4, h_r=540,
    x=dem['x'], y=dem['y'],
    lat=np.array([45.0]), lat0=45.0,
    h_grid=dem['hGrid'],
    b_mwl_sample=np.array([9.47e-3, 8.03]),
    ij_catch=ij_catch, ptr_catch=ptr_catch,
    sample_d2h=sample_d2h, sample_d18o=sample_d18o,
    cov=np.array([[1e-6, 0], [0, 1e-6]]),
    n_parameters_free=9, is_fit=False
)

d2h_pred = fitted_result[25]  # Predicted d2H at sample points
d18o_pred = fitted_result[26]  # Predicted d18O at sample points

# Plot comparison
from opi.viz import plot_sample_comparison
fig, axes = plot_sample_comparison(
    sample_d2h, sample_d18o,
    d2h_pred, d18o_pred,
    title='Fitting Results: Observed vs Predicted'
)
plt.show()

In [ ]:
# Plot residuals
from opi.viz import plot_residuals

fig, axes = plot_residuals(
    sample_d2h, sample_d18o,
    d2h_pred, d18o_pred,
    elevation=sample_elev,
    title='Residuals vs Elevation'
)
plt.show()

## 5. Save Fitted Solution

In [ ]:
# Save fitted parameters to run file (with solution)
run_data['beta'] = fitted_beta.tolist()
write_run_file('../examples/data/fitted_run.run', run_data)

print("Fitted run file saved: fitted_run.run")
print("\nYou can now use this run file for forward calculations:")
print("  result = opi.opi_calc_one_wind('fitted_run.run')")

## Next Steps

- Try fixing some parameters (e.g., set kappa=0) and re-fit
- Experiment with different sample distributions
- See `03_real_data.ipynb` for working with real field data